## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Fort Dodge,US,42.4975,-94.1680,90.01,51,0,12.66,clear sky
1,1,Babaeski,TR,41.4325,27.0931,66.09,55,0,4.50,clear sky
2,2,Cape Town,ZA,-33.9258,18.4232,57.90,82,10,5.99,clear sky
3,3,Nouadhibou,MR,20.9310,-17.0347,69.78,88,21,12.66,few clouds
4,4,Atuona,PF,-9.8000,-139.0333,77.31,72,9,14.41,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Atuona,PF,-9.8000,-139.0333,77.31,72,9,14.41,clear sky
11,11,Thompson,CA,55.7435,-97.8558,77.16,41,75,11.50,broken clouds
12,12,Meulaboh,ID,4.1363,96.1285,75.07,88,83,5.10,broken clouds
19,19,Albany,US,42.6001,-73.9662,79.36,48,65,8.37,broken clouds
20,20,Victoria,HK,22.2855,114.1577,86.38,84,78,3.00,broken clouds
26,26,Vaini,TO,-21.2000,-175.2000,80.76,89,20,10.36,few clouds
27,27,Half Moon Bay,US,37.4636,-122.4286,84.87,67,100,19.57,haze
29,29,Hithadhoo,MV,-0.6000,73.0833,82.99,75,99,13.38,overcast clouds
31,31,Monrovia,LR,6.3005,-10.7969,76.77,85,100,10.96,overcast clouds
33,33,Bathsheba,BB,13.2167,-59.5167,83.01,95,40,18.41,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 224 entries, 4 to 657
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           224 non-null    int64  
 1   City                 224 non-null    object 
 2   Country              224 non-null    object 
 3   Lat                  224 non-null    float64
 4   Lng                  224 non-null    float64
 5   Max Temp             224 non-null    float64
 6   Humidity             224 non-null    int64  
 7   Cloudiness           224 non-null    int64  
 8   Wind Speed           224 non-null    float64
 9   Current Description  224 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 19.2+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.drop(columns=['Unnamed: 0'])

clean_df = clean_df.reset_index(drop=True)

clean_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Atuona,PF,-9.8000,-139.0333,77.31,72,9,14.41,clear sky
1,Thompson,CA,55.7435,-97.8558,77.16,41,75,11.50,broken clouds
2,Meulaboh,ID,4.1363,96.1285,75.07,88,83,5.10,broken clouds
3,Albany,US,42.6001,-73.9662,79.36,48,65,8.37,broken clouds
4,Victoria,HK,22.2855,114.1577,86.38,84,78,3.00,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,77.31,clear sky,-9.8000,-139.0333,
1,Thompson,CA,77.16,broken clouds,55.7435,-97.8558,
2,Meulaboh,ID,75.07,broken clouds,4.1363,96.1285,
3,Albany,US,79.36,broken clouds,42.6001,-73.9662,
4,Victoria,HK,86.38,broken clouds,22.2855,114.1577,
5,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,
6,Half Moon Bay,US,84.87,haze,37.4636,-122.4286,
7,Hithadhoo,MV,82.99,overcast clouds,-0.6000,73.0833,
8,Monrovia,LR,76.77,overcast clouds,6.3005,-10.7969,
9,Bathsheba,BB,83.01,scattered clouds,13.2167,-59.5167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
#determine any blank hotel names
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 224 non-null    object 
 1   Country              224 non-null    object 
 2   Max Temp             224 non-null    float64
 3   Current Description  224 non-null    object 
 4   Lat                  224 non-null    float64
 5   Lng                  224 non-null    float64
 6   Hotel Name           224 non-null    object 
dtypes: float64(3), object(4)
memory usage: 12.4+ KB


In [10]:
# 7. Determine if any Hotel Name is ' '
hotel_df[hotel_df['Hotel Name'] == ''].index


Int64Index([3, 22, 27, 58, 85, 89, 121, 156, 164, 179, 185], dtype='int64')

In [11]:
#replace ' ' with NaN and drop NaN rows
import numpy as np

hotel_df.replace(r'^\s*$', np.nan, regex=True, inplace = True)

clean_hotel_df = hotel_df.dropna()

clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,77.31,clear sky,-9.8000,-139.0333,Villa Enata
1,Thompson,CA,77.16,broken clouds,55.7435,-97.8558,Burntwood Hotel
2,Meulaboh,ID,75.07,broken clouds,4.1363,96.1285,Kost Pria
4,Victoria,HK,86.38,broken clouds,22.2855,114.1577,Mini Central
5,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,Keleti Beach Resort
6,Half Moon Bay,US,84.87,haze,37.4636,-122.4286,"The Ritz-Carlton, Half Moon Bay"
7,Hithadhoo,MV,82.99,overcast clouds,-0.6000,73.0833,Scoop Guest House
8,Monrovia,LR,76.77,overcast clouds,6.3005,-10.7969,Sunset Inn
9,Bathsheba,BB,83.01,scattered clouds,13.2167,-59.5167,Atlantis Hotel
10,Jamestown,US,76.59,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown


In [12]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</d>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))